# Support vector machines

In [2]:
# import libraries

import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns

from matplotlib.backends.backend_pdf import PdfPages
from sklearn.decomposition import PCA

from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.feature_selection import GenericUnivariateSelect, mutual_info_classif, mutual_info_regression, f_regression

from sklearn import preprocessing
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn import linear_model

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics.pairwise import pairwise_kernels

from sklearn import model_selection

from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

## Data pre-processing

In [7]:
# load training data

# load data from csv file
df_train_features = pd.read_csv ('train_features.csv')
df_train_labels = pd.read_csv('train_labels.csv')

# Load test data
df_test_features = pd.read_csv ('test_features.csv')

### Sorting labels

In [8]:
df_train_labels = df_train_labels.sort_values('pid')
df_train_features = df_train_features.sort_values(['pid', 'Time'])

# # Droping time
# df_train_features = df_train_features.drop('Time', axis = 1)
# df_test_features = df_test_features.drop('Time', axis = 1)

# Using TSFresh

In [ ]:
df_train_features

 ### Histogram of the output labels 

We should check for class imbalance.

In [ ]:
df_train_labels.hist()

# with PdfPages("./Results/Labels_histogram.pdf") as export_pdf:
#     for i in list(df_train_labels)[1:]:
#         df_train_labels.hist(column = i, bins = 100)
#         export_pdf.savefig()

One can see the class imbalance problem here. Other observations:
  * Heartrate, RRate, ABPm,  distribution is similar to a normal distribution
  * SpO2 is like a censored normal distribution. 
  * For all of the other features, class imbalance is an obvious problem.

A basic strategy that could be used here: Upsample both classes! Do the upsampling efficiently, not just replicating the datapoints

### Boxplot over features

In [ ]:
# data inspection: 
#############################################
# range of the provided data?
print(df_train_features.agg([min, max]))

# Boxplotting the data
# fig2, ax2 = plt.subplots()
# ax2.set_title('BUN')
# ax2.boxplot(df_train_features.iloc[:,5], notch=True)

plt.figure(figsize=(16, 16))
ax = sns.boxplot(data = df_train_features.iloc[:,1:])
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=90,
    horizontalalignment='right'
);

# with PdfPages("./Results/Train_columns_boxplot.pdf") as export_pdf:
#     for i in list(df_train_labels)[1:]:
#         df_train_labels.hist(column = i, bins = 100)
#         export_pdf.savefig()

In [ ]:
# calculate the correlation matrix
corr = df_train_features.corr()

# plot the heatmap
plt.figure(figsize=(16, 16))
ax = sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns, 
        vmin=-1, vmax=1, center=0, 
           cmap=sns.diverging_palette(20, 220, n=200))
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

### Visualizing pattern of missing values

In [ ]:
# how much missing data? 
print("Percentage of missing values:")
print(df_train_features.isnull().sum(axis=0) / len(df_train_features))

msno.matrix(df_train_features)

# Plotting the correlation between the missing values
msno.heatmap(df_train_features)

### Train data pre-processing

In [9]:
# Which columns have less than a percent NA
NA_percent = 0.8
NA_percent_severe = 0.91

sel_features = df_train_features.isnull().sum(axis=0) < (NA_percent * df_train_features.shape[0])
inds = np.where(sel_features == True)

sel_features_2 = (df_train_features.isnull().sum(axis=0) < (NA_percent_severe * df_train_features.shape[0])) & (df_train_features.isnull().sum(axis=0) > (NA_percent * df_train_features.shape[0]))        
inds_2 = np.where(sel_features_2 == True)

sel_features_3 = df_train_features.isnull().sum(axis=0) > (NA_percent_severe * df_train_features.shape[0])
inds_3 = np.where(sel_features_3 == True)

print(inds[0])
print(inds_2[0])
print(inds_3[0])

present_columns = df_train_features.iloc[:,inds[0]]

present_columns_agg = present_columns.groupby('pid').agg([np.min, np.max, np.mean, np.median, np.std])
present_columns_agg = present_columns_agg.drop(present_columns_agg.columns[[0,1,3,4]], axis=1)

missing_columns = df_train_features.iloc[:,np.append(0,inds_2)]

missing_columns_agg = missing_columns.groupby('pid').agg([np.min, np.max, np.mean])

missing_columns_severe = df_train_features.iloc[:,np.append(0,inds_3)]

missing_columns_agg_severe = missing_columns_severe.groupby('pid').agg(np.mean)


print(present_columns_agg.shape)
print(missing_columns_agg.shape)
print(missing_columns_agg_severe.shape)

[ 0  1  2  7 11 21 22 25 28 32 35]
[ 8 16 18 24 31 36]
[ 3  4  5  6  9 10 12 13 14 15 17 19 20 23 26 27 29 30 33 34]
(18995, 46)
(18995, 18)
(18995, 20)


In [10]:
df_train_agg_features = pd.merge(present_columns_agg, missing_columns_agg, on="pid")
df_train_agg_features = pd.merge(df_train_agg_features, missing_columns_agg_severe, on = "pid")
print(df_train_agg_features.shape)
print(df_train_agg_features.columns)

(18995, 84)
Index([       ('Time', 'mean'),         ('Age', 'amin'),
               ('Age', 'amax'),         ('Age', 'mean'),
             ('Age', 'median'),          ('Age', 'std'),
              ('Temp', 'amin'),        ('Temp', 'amax'),
              ('Temp', 'mean'),      ('Temp', 'median'),
               ('Temp', 'std'),       ('RRate', 'amin'),
             ('RRate', 'amax'),       ('RRate', 'mean'),
           ('RRate', 'median'),        ('RRate', 'std'),
           ('Glucose', 'amin'),     ('Glucose', 'amax'),
           ('Glucose', 'mean'),   ('Glucose', 'median'),
            ('Glucose', 'std'),        ('ABPm', 'amin'),
              ('ABPm', 'amax'),        ('ABPm', 'mean'),
            ('ABPm', 'median'),         ('ABPm', 'std'),
              ('ABPd', 'amin'),        ('ABPd', 'amax'),
              ('ABPd', 'mean'),      ('ABPd', 'median'),
               ('ABPd', 'std'),        ('SpO2', 'amin'),
              ('SpO2', 'amax'),        ('SpO2', 'mean'),
            ('SpO2'

In [ ]:
# df_train_agg_features = df_train_features.groupby('pid').agg([np.min, np.max, np.mean np.std])
# df_train_agg_features = df_train_agg_features.iloc[:,5:]
# # Removing ETCo2 mean and max since it has so many NA
# df_train_agg_features = df_train_agg_features.drop(df_train_agg_features.columns[[2,3]],  axis = 1)
# print(df_train_agg_features.columns)
# df_train_agg_features.columns
# print(int(df_train_agg_features.shape[1]))
# print(int(df_train_agg_features.shape[1]/3))

# # how much missing data? 
# print("number of missing values:")
# print(df_train_agg_features.isnull().sum(axis=0))

# na_percent_max = int(0.8 * df_train_agg_features.shape[0])
# tmp = pd.DataFrame(df_train_agg_features)
# for i in range(1, (int(df_train_agg_features.shape[1]/3))):
#     na_count = df_train_agg_features.iloc[:,i].isna().sum()
#     print(df_train_agg_features.columns[i])
#     print(na_count)
    
#     if(na_count > na_percent_max):
#         print("should be removed")


In [ ]:
# Run the imputer with a simple Random Forest estimator
imp = IterativeImputer(RandomForestRegressor(n_estimators=5), max_iter=10, random_state=1)

#perform filling
df_train_agg_imputed_features = pd.DataFrame(imp.fit_transform(df_train_agg_features), columns=df_train_agg_features.columns)

In [ ]:
# impute missing data points
#imp = SimpleImputer(strategy="mean")
imputer = KNNImputer(n_neighbors = 10)
#imputer = IterativeImputer(random_state=0, verbose = 2, max_iter = 30)
df_train_agg_imputed_features = imputer.fit_transform(df_train_agg_features)
#print(df_train_agg_imputed_features)

In [ ]:
# scale the data
min_max_scaler = preprocessing.StandardScaler()
# standard_scalar = preprocessing.StandardScaler()

data_train_scaled = min_max_scaler.fit_transform(df_train_agg_imputed_features)

In [ ]:
# REARRANGE THE LABELS, TO MATCH THE REARRANGED FEATURES
df_train_labels_sorted = df_train_labels.sort_values(by = 'pid')
print(df_train_labels_sorted[['pid']])
print(df_train_labels[['pid']])
print(df_train_agg_features)

In [ ]:
# Visualizing the training data after imputing and aggregating

plt.figure(figsize=(16, 16))
ax = sns.boxplot(data = pd.DataFrame(data_train_scaled))
ax.set_xticklabels(
    list(df_train_features),
    rotation=90,
    horizontalalignment='right'
);

In [ ]:
# What is the correlation between the 
pd.DataFrame(data_train_scaled).corrwith(other = pd.DataFrame(df_train_agg_imputed_features), method = "spearman").transpose()

### PCA plot 

In [ ]:
pca = PCA(n_components=2)

principalComponents = pca.fit_transform(data_train_scaled)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

finalDf = pd.concat([principalDf, df_train_labels[[df_train_labels.columns[11]]]], axis = 1)

fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA for label i', fontsize = 20)
targets = [0, 1]
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf[df_train_labels.columns[11]] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

### Test Data pre-processing

In [ ]:
# data inspection: 
#############################################
# range of the provided data?
print(df_test_features.agg([min, max]))

# how much missing data? 
print("number of missing values:")
print(df_test_features.isnull().sum(axis=0))

In [11]:
# We should use the same indices as before
print(inds[0])
print(inds_2[0])
print(inds_3[0])

present_columns = df_test_features.iloc[:,inds[0]]

present_columns_agg = present_columns.groupby('pid').agg([np.min, np.max, np.mean, np.median, np.std])
present_columns_agg = present_columns_agg.drop(present_columns_agg.columns[[0,1,3,4]], axis=1)

missing_columns = df_test_features.iloc[:,np.append(0,inds_2)]

missing_columns_agg = missing_columns.groupby('pid').agg([np.min, np.max, np.mean])

missing_columns_severe = df_test_features.iloc[:,np.append(0,inds_3)]

missing_columns_agg_severe = missing_columns_severe.groupby('pid').agg(np.mean)

[ 0  1  2  7 11 21 22 25 28 32 35]
[ 8 16 18 24 31 36]
[ 3  4  5  6  9 10 12 13 14 15 17 19 20 23 26 27 29 30 33 34]


In [12]:
df_test_agg_features = pd.merge(present_columns_agg, missing_columns_agg, on="pid")
df_test_agg_features = pd.merge(df_test_agg_features, missing_columns_agg_severe, on = "pid")
print(df_test_agg_features.shape)
print(df_test_agg_features.columns)

(12664, 84)
Index([       ('Time', 'mean'),         ('Age', 'amin'),
               ('Age', 'amax'),         ('Age', 'mean'),
             ('Age', 'median'),          ('Age', 'std'),
              ('Temp', 'amin'),        ('Temp', 'amax'),
              ('Temp', 'mean'),      ('Temp', 'median'),
               ('Temp', 'std'),       ('RRate', 'amin'),
             ('RRate', 'amax'),       ('RRate', 'mean'),
           ('RRate', 'median'),        ('RRate', 'std'),
           ('Glucose', 'amin'),     ('Glucose', 'amax'),
           ('Glucose', 'mean'),   ('Glucose', 'median'),
            ('Glucose', 'std'),        ('ABPm', 'amin'),
              ('ABPm', 'amax'),        ('ABPm', 'mean'),
            ('ABPm', 'median'),         ('ABPm', 'std'),
              ('ABPd', 'amin'),        ('ABPd', 'amax'),
              ('ABPd', 'mean'),      ('ABPd', 'median'),
               ('ABPd', 'std'),        ('SpO2', 'amin'),
              ('SpO2', 'amax'),        ('SpO2', 'mean'),
            ('SpO2'

In [ ]:
# # # aggregate data for each pid
# # df_test_aggregate_features = df_test_features.groupby('pid').agg('median')

# df_test_agg_features = df_test_features.groupby('pid').agg([np.min, np.max, np.mean])

# df_test_agg_features = df_test_agg_features.iloc[:,5:]
# # Removing ETCo2 mean and max since it has so many NA
# df_test_agg_features = df_test_agg_features.drop(df_test_agg_features.columns[[2,3]],  axis = 1)

In [ ]:
# impute missing data points
# should we impute it with the same imputer that we've used for train?

imputer = KNNImputer(n_neighbors= 10)
#imputer = IterativeImputer(random_state=0, verbose = 1)
df_test_agg_imputed_features = imputer.fit_transform(df_test_agg_features)

In [ ]:
# scale test data
min_max_scaler = preprocessing.StandardScaler()
data_test_scaled = min_max_scaler.fit_transform(df_test_agg_imputed_features)

In [ ]:
# This part is for chanding the order of standardizing and imputing

# # scale the data
# min_max_scaler = preprocessing.StandardScaler()
# # standard_scalar = preprocessing.StandardScaler()

# data_train_scaled_tmp = min_max_scaler.fit_transform(df_train_agg_features)
# imputer = KNNImputer(n_neighbors = 5)
# #imputer = IterativeImputer(random_state=0, verbose = 2, max_iter = 30)
# data_train_scaled = imputer.fit_transform(data_train_scaled_tmp)


# min_max_scaler = preprocessing.StandardScaler()
# # standard_scalar = preprocessing.StandardScaler()

# data_test_scaled_tmp = min_max_scaler.fit_transform(df_test_agg_features)
# imputer = KNNImputer(n_neighbors = 5)
# #imputer = IterativeImputer(random_state=0, verbose = 2, max_iter = 30)
# data_test_scaled = imputer.fit_transform(data_test_scaled_tmp)

In [ ]:
pd.DataFrame(data_train_scaled).to_csv("./Results/dat_train_scaled.csv", index = False)
pd.DataFrame(data_test_scaled).to_csv("./Results/dat_test_scaled.csv", index = False)

In [ ]:
# If want to run from here:
data_train_scaled = pd.read_csv("./Results/dat_train_scaled.csv")
data_test_scaled = pd.read_csv("./Results/dat_test_scaled.csv")

## Fit a model & Predict

### predict with support vector machine classification and use probabilities

In [ ]:
# first for the labels that have an output [0,1]
test_pids = list(set(df_test_features.pid))
columns_1 = [test_pids]

from sklearn.utils import resample

for i in range(1, 12):
    
    # feature selection
    transformer =  GenericUnivariateSelect(score_func= mutual_info_classif, mode ='k_best', param=40)
    train_features = pd.DataFrame(transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i]))
    print("For feature ", df_train_labels.columns[i])
#     print(df_train_agg_features.columns[transformer.get_support(indices = True)])
    test_features = pd.DataFrame(transformer.transform(data_test_scaled))

#     values_1 = train_features.loc[df_train_labels[df_train_labels.columns[i]] == 1]
#     values_0 = train_features.loc[df_train_labels[df_train_labels.columns[i]] == 0]
#     values_0 = resample(values_0, replace = False, n_samples = values_1.shape[0])

#     train_features = pd.concat([values_0, values_1])
    
#     labels = np.repeat([0,1], values_0.shape[0])
    
    #clf = BaggingClassifier(SVC(kernel = 'poly', degree = 5, class_weight = 'balanced', verbose = True, C = 10))
    clf_w = SVC(kernel = 'rbf', class_weight = 'balanced', verbose = 2)
    
    parameters = {'C':(0.1, 1, 5, 10)}
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 5,
                                        refit = True, scoring = 'roc_auc', verbose = 2,
                                       n_jobs=6, return_train_score = True)
    clf.fit(train_features, df_train_labels.iloc[:,i])
#     clf.fit(train_features, labels)
    
#     print(clf.cv_results_)
    print(clf.best_params_)
    print(clf.best_score_)
    # compute probabilites as opposed to predictions
    #dual_coefficients = clf.dual_coef_    # do we have to normalize with norm of this vector ?
    
    distance_hyperplane = clf.decision_function(test_features)
    probability = np.empty(len(distance_hyperplane))
    for j in range(0, len(probability)):
        if distance_hyperplane[j] < 0:
            probability[j] = 1 - 1/(1 + math.exp(distance_hyperplane[j]))
        else:
            probability[j] = 1/(1 + math.exp(-distance_hyperplane[j]))
    columns_1.append(probability)

    
    distance_hyperplace_train = clf.decision_function(train_features)
    probability = np.empty(len(distance_hyperplace_train))
    for j in range(0, len(probability)):
        if distance_hyperplace_train[j] < 0:
            probability[j] = 1 - 1/(1 + math.exp(distance_hyperplace_train[j]))
        else:
            probability[j] = 1/(1 + math.exp(-distance_hyperplace_train[j]))
    
    tmp = roc_auc_score(y_score= probability, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)


In [ ]:
# labels that have a real value
columns_2 = []
# from sklearn.kernel_ridge import KernelRidge

for i in range(12, 16):
    # feature selection
    transformer =  GenericUnivariateSelect(score_func= mutual_info_regression, mode ='k_best', param = 75)
    train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
#     print(df_train_agg_features.columns[transformer.get_support(indices = True)])
    test_features = transformer.transform(data_test_scaled)
    
    clf_w = SVR(kernel = 'rbf', cache_size = 6000)
# #     clf_w = NuSVR(nu=0.5, kernel = 'linear')
    parameters = {'C':(0.1, 1,10)}
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 10,
                                       refit = True, scoring = 'r2', verbose = 2, n_jobs=6)
#     clf = KernelRidge(kernel = 'poly', degree = 5)
#     parameters = {'alpha':(0.1,1,10,30)}
#     clf = model_selection.GridSearchCV(estimator= clf, param_grid = parameters, cv = 3,
#                                       refit = True, scoring = 'r2', verbose = 2, n_jobs=6)
    clf.fit(train_features, df_train_labels.iloc[:,i])
    
#     print(clf.cv_results_)
    print(clf.best_params_)
    print(clf.best_score_)

    pred_train = clf.predict(train_features)
    tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
    print("R2 for feature", list(df_train_labels)[i] , " : ", tmp)
    
    pred = clf.predict(test_features)
    columns_2.append(pred)
    

In [ ]:
columns_final = columns_1 + columns_2

### predict with Support vector regression and then compute sigmoid function

In [ ]:
# first for the labels that have an output [0,1]

# columns_1 = [test_pids]

# for i in range(1,12):
    
#     clf = SVR(kernel = 'poly', degree = 3, max_iter = 10000)
#     clf.fit(data_train_scaled, df_train_labels.iloc[:,i])
#     pred = clf.predict(data_test_scaled)
#     prob = np.empty(len(pred))
#     for j in range(0, len(pred)):
#         prob[j] = 1 / (1 + math.exp(-pred[j]))
#     columns_1.append(prob)
    
#     pred_train = clf.predict(data_train_scaled)
#     prob_train = np.empty(len(pred_train))
#     for j in range(0, len(pred_train)):
#         prob_train[j] = 1 / (1 + math.exp(-pred_train[j]))    
#     tmp = roc_auc_score(y_score= prob_train, y_true= df_train_labels.iloc[:,i])
#     print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)


In [ ]:
# #labels that have a real value

# columns_2 = []

# for i in range(12, 16):
    
#     # feature selection
#     transformer =  GenericUnivariateSelect(score_func= mutual_info_regression, mode ='k_best', param=20)
#     train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
#     print(list(data_train_scaled)[transformer.get_support()])
#     test_features = transformer.transform(data_test_scaled)
    

#     clf_w = LinearSVR()
#     parameters = {'C':(0.1,1,10,30,60,100)}
#     clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 2,
#                                        refit = True, scoring = 'r2', verbose = 1, n_jobs=6)
#     clf.fit(train_features, df_train_labels.iloc[:,i])
    
#     print(clf.cv_results_)
#     pred = clf.predict(test_features)
#     columns_2.append(pred)
    
#     pred_train = clf.predict(train_features)
#     tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
#     print("R2 for feature", list(df_train_labels)[i] , " : ", tmp)

In [ ]:
transformer =  GenericUnivariateSelect(score_func= mutual_info_regression, mode ='k_best', param=20)
train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,11])
test_features = transformer.transform(data_test_scaled)

In [ ]:
df_train_agg_features.columns[transformer.get_support(indices = True)]

In [ ]:
columns_final = columns_1 + columns_2

### Random forest

In [ ]:
# Random forest Classifier
columns_1 = [test_pids]
for i in range(1, 12):
    clf = RandomForestClassifier(min_samples_leaf=2, class_weight='balanced', oob_score=False, bootstrap=False)
    clf.fit(data_train_scaled, df_train_labels.iloc[:,i])
    print(clf.oob_score)
    # compute probabilites as opposed to predictions
    probability = clf.apply(data_test_scaled)
    probs = [i[1] for i in probability] 
    columns_1.append(probs)
    
    
    probability = clf.predict_proba(data_train_scaled)

    probs = [i[1] for i in probability]            
    tmp = roc_auc_score(y_score= probs, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)

# Kernelized Logistic Regression

In [ ]:
# first for the labels that have an output [0,1]
test_pids = list(set(df_test_features.pid))
columns_1 = [test_pids]

from sklearn.linear_model import LogisticRegression
from sklearn.kernel_approximation import Nystroem
from sklearn.model_selection import StratifiedKFold, KFold

for i in range(1, 12):

    feature_map_nystroem = Nystroem(kernel = 'rbf',
                                 random_state=1,
                                 n_components=500)
    
    train_transformed = feature_map_nystroem.fit_transform(data_train_scaled)
    test_transformed = feature_map_nystroem.transform(data_test_scaled)
    
    clf_w = LogisticRegression(penalty = 'l1', class_weight = 'balanced', max_iter=10000, solver = 'saga')
    
    # checked before
    #parameters = {'alpha':(0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 20, 30)}
    parameters = {'C':(0.1, 1, 5, 10, 20)}
    
    skf = StratifiedKFold(n_splits=10, shuffle = True, random_state = 1)
    
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = skf,
                                        refit = True, scoring = 'roc_auc', verbose = 1,
                                       n_jobs=6, return_train_score = True)
    clf.fit(train_transformed, df_train_labels.iloc[:,i])
    
    print(clf.cv_results_['mean_train_score'])
    print(clf.cv_results_['mean_test_score'])
    print(clf.best_params_)
    print(clf.best_score_)
 
    probability_tmp = clf.predict_proba(test_transformed)
    probability = [item[1] for item in probability_tmp]
    columns_1.append(probability)

    probability_tmp = clf.predict_proba(train_transformed)
    probability_train = [item[1] for item in probability_tmp]
    tmp = roc_auc_score(y_score= probability_train, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)
    

# Compute the kernel and use SGD Classifier and Regressor

In [ ]:
# first for the labels that have an output [0,1]
test_pids = list(set(df_test_features.pid))
columns_1 = [test_pids]

# from sklearn.kernel_ridge import KernelRidge
from sklearn.kernel_approximation import Nystroem
from sklearn import linear_model


for i in range(1, 12):
   
    # feature selection
    transformer =  GenericUnivariateSelect(score_func= mutual_info_classif, mode ='k_best', param=70)
    train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
    print("For feature ", df_train_labels.columns[i])
#     print(df_train_agg_features.columns[transformer.get_support(indices = True)])
    test_features = transformer.transform(data_test_scaled)

    
    feature_map_nystroem = Nystroem(kernel = 'poly', degree = 3,
                                 random_state=1,
                                 n_components=300)
    train_transformed = feature_map_nystroem.fit_transform(train_features)
    test_transformed = feature_map_nystroem.transform(test_features)
    
    clf_w = linear_model.SGDClassifier(max_iter=100000, tol=1e-4, penalty = "l2", 
                                       loss = "epsilon_insensitive", class_weight='balanced')
    # checked before
    #parameters = {'alpha':(0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 20, 30)}
    parameters = {'alpha':(0.1, 1, 5, 10)}
    
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 10,
                                        refit = True, scoring = 'roc_auc', verbose = 1,
                                       n_jobs=6, return_train_score = True)
    clf.fit(train_transformed, df_train_labels.iloc[:,i])
    
#     print(clf.cv_results_)
    print(clf.best_params_)
    print(clf.best_score_)
    # compute probabilites as opposed to predictions
    #dual_coefficients = clf.dual_coef_    # do we have to normalize with norm of this vector ?
    
    distance_hyperplane = clf.decision_function(test_transformed)
    probability = np.empty(len(distance_hyperplane))
    for j in range(0, len(probability)):
        if distance_hyperplane[j] < 0:
            probability[j] = 1 - 1/(1 + math.exp(distance_hyperplane[j]))
        else:
            probability[j] = 1/(1 + math.exp(-distance_hyperplane[j]))
    columns_1.append(probability)

    
    distance_hyperplace_train = clf.decision_function(train_transformed)
    probability = np.empty(len(distance_hyperplace_train))
    for j in range(0, len(probability)):
        if distance_hyperplace_train[j] < 0:
            probability[j] = 1 - 1/(1 + math.exp(distance_hyperplace_train[j]))
        else:
            probability[j] = 1/(1 + math.exp(-distance_hyperplace_train[j]))
    
    tmp = roc_auc_score(y_score= probability, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)
    

In [ ]:
# labels that have a real value
columns_2 = []

for i in range(12, 16):
#     feature selection
#     transformer =  GenericUnivariateSelect(score_func= mutual_info_regression, mode ='k_best', param = 70)
#     train_features = transformer.fit_transform(data_train_scaled, df_train_labels.iloc[:,i])
# #     print(df_train_agg_features.columns[transformer.get_support(indices = True)])
#     test_features = transformer.transform(data_test_scaled)
    
    feature_map_nystroem = Nystroem(kernel = 'rbf',
                                 random_state=1,
                                 n_components=500)
    train_features = feature_map_nystroem.fit_transform(data_train_scaled)
    test_features = feature_map_nystroem.transform(data_test_scaled)
    
    clf_w = linear_model.SGDRegressor(max_iter=100000, tol=1e-4,
                                     loss = 'epsilon_insensitive', penalty = 'l1')
    parameters = {'alpha':(0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 20, 30)}
    clf = model_selection.GridSearchCV(estimator= clf_w, param_grid = parameters, cv = 10,
                                       refit = True, scoring = 'r2', verbose = 1, n_jobs=6, return_train_score=True)
#     clf = KernelRidge(kernel = 'poly', degree = 5)
#     parameters = {'alpha':(0.1,1,10,30)}
#     clf = model_selection.GridSearchCV(estimator= clf, param_grid = parameters, cv = 3,
#                                       refit = True, scoring = 'r2', verbose = 2, n_jobs=6)
    clf.fit(train_features, df_train_labels.iloc[:,i])
    
#     print(clf.cv_results_)
    print(clf.cv_results_['mean_train_score'])
    print(clf.cv_results_['mean_test_score'])
    print(clf.best_params_)
    print(clf.best_score_)
    
    pred = clf.predict(test_features)
    columns_2.append(pred) 

    pred_train = clf.predict(train_features)
    tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
    print("R2 for feature", list(df_train_labels)[i] , " : ", tmp)
   

In [ ]:
columns_final = columns_1 + columns_2

# XGBoosting

The benefit of XGBoost is that it accepts missing values.

In [13]:
import xgboost as xgb
# simple example
# load file from text file, also binary buffer generated by xgboost
dtrain = xgb.DMatrix(df_train_agg_features, label=df_train_labels.iloc[:,11])
dtest = xgb.DMatrix(df_test_agg_features)

In [14]:
param = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic', 'verbosity':1}
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist = [(dtrain, 'train')]

In [15]:
num_round = 10
xgb.cv(param, dtrain, num_round, nfold=10,
       metrics={'auc'}, seed=0,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])
# bst = xgb.train(param, dtrain, num_round)
# pred = bst.predict(dtest)

[0]	train-auc:0.64354+0.00724	test-auc:0.63346+0.02823
[1]	train-auc:0.69151+0.00626	test-auc:0.67614+0.03512
[2]	train-auc:0.70502+0.00350	test-auc:0.68443+0.03363
[3]	train-auc:0.71608+0.00508	test-auc:0.69292+0.02595
[4]	train-auc:0.72693+0.00470	test-auc:0.70031+0.02049
[5]	train-auc:0.73649+0.00484	test-auc:0.70530+0.02197
[6]	train-auc:0.74349+0.00496	test-auc:0.70955+0.02387
[7]	train-auc:0.75035+0.00495	test-auc:0.71195+0.02153
[8]	train-auc:0.75524+0.00404	test-auc:0.71086+0.02351
[9]	train-auc:0.75904+0.00309	test-auc:0.70961+0.02207


,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.643538,0.007237,0.633455,0.028234
1,0.691513,0.006258,0.676137,0.035120
2,0.705022,0.003501,0.684427,0.033634
3,0.716080,0.005076,0.692920,0.025947
4,0.726930,0.004701,0.700306,0.020486
5,0.736486,0.004842,0.705297,0.021974
6,0.743489,0.004956,0.709551,0.023865
7,0.750346,0.004955,0.711953,0.021528
8,0.755236,0.004044,0.710862,0.023507
9,0.759035,0.003087,0.709613,0.022069


In [35]:
from sklearn.model_selection import KFold

test_pids = list(set(df_test_features.pid))
columns_1 = [test_pids]

kf = KFold(n_splits=10, shuffle=True, random_state=1)

for i in range(1,12):

    clf_w = xgb.XGBClassifier(cv = 10)
        
    clf_w.fit(df_train_agg_features, df_train_labels.iloc[:,i])
    
    dtrain = xgb.DMatrix(df_train_agg_features, label=df_train_labels.iloc[:,i])
    num_round = 10
    xgb.cv(param, dtrain, num_round, nfold=10,
       metrics={'auc'}, seed=0,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])
    
    probability = clf_w.predict(df_test_agg_features)
    columns_1.append(probability)

    probability_train = clf_w.predict(df_train_agg_features)
#     probability_train = [item[1] for item in probability_tmp]
    tmp = roc_auc_score(y_score= probability_train, y_true= df_train_labels.iloc[:,i])
    print("ROC AUC for feature", list(df_train_labels)[i] , " : ", tmp)

[0]	train-auc:0.88610+0.00722	test-auc:0.88471+0.01329
[1]	train-auc:0.90073+0.00176	test-auc:0.89910+0.00653
[2]	train-auc:0.90932+0.00234	test-auc:0.90496+0.00716
[3]	train-auc:0.91530+0.00239	test-auc:0.91091+0.00799
[4]	train-auc:0.91818+0.00088	test-auc:0.91331+0.00677
[5]	train-auc:0.92019+0.00093	test-auc:0.91476+0.00664
[6]	train-auc:0.92169+0.00097	test-auc:0.91593+0.00663
[7]	train-auc:0.92278+0.00091	test-auc:0.91635+0.00752
[8]	train-auc:0.92399+0.00107	test-auc:0.91699+0.00808
[9]	train-auc:0.92498+0.00116	test-auc:0.91749+0.00821
ROC AUC for feature LABEL_BaseExcess  :  0.9737049936280611
[0]	train-auc:0.74258+0.00134	test-auc:0.74033+0.01356
[1]	train-auc:0.76549+0.00436	test-auc:0.75907+0.01270
[2]	train-auc:0.77302+0.00434	test-auc:0.76308+0.01513
[3]	train-auc:0.77888+0.00413	test-auc:0.76977+0.01801
[4]	train-auc:0.78824+0.00418	test-auc:0.77541+0.02293
[5]	train-auc:0.79508+0.00335	test-auc:0.78146+0.02379
[6]	train-auc:0.79966+0.00338	test-auc:0.78441+0.02409
[7]	t

In [ ]:
columns_2 = []
param = {'max_depth': 2, 'eta': 1, 'objective': 'reg:squarederror', 'verbosity':1}
param['nthread'] = 4
param['eval_metric'] = 'rmse'
evallist = [(dtrain, 'train')]

for i in range(12,16):

    clf_w = xgb.XGBRegressor(cv = 10)
        
    clf_w.fit(df_train_agg_features, df_train_labels.iloc[:,i])
    
    dtrain = xgb.DMatrix(df_train_agg_features, label=df_train_labels.iloc[:,i])
    num_round = 10
    xgb.cv(param, dtrain, num_round, nfold=10, seed=0,
       callbacks=[xgb.callback.print_evaluation(show_stdv=True)])
    
    probability = clf_w.predict(df_test_agg_features)
    columns_1.append(probability)

    pred_train = clf_w.predict(df_train_agg_features)
#     probability_train = [item[1] for item in probability_tmp]
    tmp = r2_score(y_pred= pred_train, y_true=df_train_labels.iloc[:,i])
    print("R2 for feature", list(df_train_labels)[i] , " : ", tmp)
   

[0]	train-rmse:2.84320+0.00569	test-rmse:2.85736+0.05203
[1]	train-rmse:2.79428+0.00671	test-rmse:2.81488+0.05538
[2]	train-rmse:2.77582+0.00709	test-rmse:2.81021+0.05452
[3]	train-rmse:2.75927+0.00829	test-rmse:2.80433+0.05643
[4]	train-rmse:2.74522+0.00924	test-rmse:2.79285+0.06114


In [32]:
columns_final = columns_1 + columns_2

[[0, 3, 5, 7, 9, 11, 12, 15, 16, 17, 19, 21, 22, 25, 28, 32, 36, 40, 42, 44, 46, 52, 58, 59, 60, 65, 66, 67, 68, 72, 75, 78, 79, 82, 83, 84, 86, 93, 95, 98, 103, 106, 108, 110, 116, 117, 118, 119, 121, 125, 126, 132, 133, 135, 139, 147, 149, 152, 155, 156, 162, 167, 169, 172, 174, 178, 181, 183, 185, 186, 189, 192, 193, 196, 197, 198, 210, 217, 218, 219, 220, 221, 223, 229, 233, 234, 236, 242, 244, 246, 248, 250, 252, 255, 259, 260, 263, 264, 267, 269, 270, 273, 280, 281, 284, 291, 293, 298, 301, 303, 304, 306, 307, 310, 312, 313, 316, 317, 318, 323, 329, 335, 336, 337, 340, 341, 345, 350, 351, 353, 356, 358, 360, 363, 368, 369, 378, 379, 385, 386, 388, 389, 390, 392, 394, 399, 404, 411, 412, 413, 414, 417, 422, 423, 425, 429, 431, 432, 433, 436, 440, 442, 447, 450, 451, 454, 455, 457, 459, 461, 465, 470, 472, 474, 476, 482, 483, 489, 495, 498, 501, 505, 510, 511, 512, 514, 515, 516, 522, 523, 525, 526, 528, 529, 530, 533, 535, 539, 543, 544, 547, 550, 551, 554, 555, 563, 566, 567, 572

## Save predictions

In [31]:
print(np.shape(columns_final))
result = pd.DataFrame(columns_final).transpose()
result.columns = list(df_train_labels)
result.to_csv('./Results/prediction.csv.zip', index=False, float_format='%.3f', compression='zip')

(18, 12664)


ValueError: Length mismatch: Expected axis has 18 elements, new values have 16 elements

In [ ]:
result.to_csv('./Results/prediction.csv', index=False, float_format='%.3f')